<a href="https://colab.research.google.com/github/SteffiJF/wine-specialization-project/blob/main/Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Processing the data and making two files, one with aggregated sales per district and one with the top five wines per district

In [1]:
!pip install "dask[dataframe]" 
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import dask.dataframe as dd
import numpy as np
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials	
from datetime import datetime
%matplotlib inline
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from datetime import datetime
from matplotlib import pyplot
from numpy import array
import seaborn as sns
import matplotlib._color_data as mcd

# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(10, 5)})

     |████████████████████████████████| 102kB 4.2MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Downloading data from Google Disk (you need to upload the files into your own Google Disk and update to your own id or choose an alternative method to load data)

In [2]:
#Settting up connection to Google Disk
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Downloading sales data
download = drive.CreateFile({'id': '16d4KjwOcHkZJaBp4C1w1PhFHuInIRuu3'})
download.GetContentFile('Salg.csv')
sales = pd.read_csv("Salg.csv", sep=';',  parse_dates = [['År', 'Måned']], low_memory=False)

#Downloading ranking data
download2 = drive.CreateFile({'id': '1rtnYjvN7q8reK-RCX3H_7qVYW_837t9J'})
download2.GetContentFile('Rangering.csv')
ranks = pd.read_csv("Rangering.csv", sep=';',  parse_dates = [['År', 'Måned']], low_memory=False)

#Downloading product data
download3 = drive.CreateFile({'id': '1s8afH0Xt_YqCpMvCuWkUeI9T0RJVuT32'})
download3.GetContentFile('Products.csv')
products = pd.read_csv("Products.csv", sep=';', low_memory=False)


Setting up data for modeling

In [4]:
wineSales = sales[sales['Varetype']=='Rødvin']
countries = wineSales['Land'].tolist()
countries = list(dict.fromkeys(countries))

totalLiters=wineSales.groupby(['Land', 'Distrikt'])['Liter denne måned i år'].sum()
totalLiters=totalLiters.to_dict()


#Making a list of tuples containing country and district, only taking the
#districts with more than 50000 liters sold in total
areas=[]
for c in countries:
  dfCountry = sales[sales['Land']== c]
  districts = sales['Distrikt'].tolist()
  districts = list(dict.fromkeys(districts))
  for d in districts:
    if (c,d) in totalLiters:
      if not pd.isnull(d):
        if totalLiters[c,d]>=50000:
          if d != 'ZZZ' and d!='RHO':
            area = (c,d)
            areas.append(area)


Constructing two new dataframes (aggregated sales per district and top 5 wines)

In [5]:
#Selecting area and features to study

country = 'ITA'  
district = 'PIE' 
product = 'Rødvin'
packaging = 'Engangsflaske av glass'

###############################################################################

# Merging files into single dataframe
def merging(sales, ranks, products, country, district, product, packaging):

  #Removing irrelevant areas
  chosenSales = sales[sales['Land']==country]
  chosenSales = chosenSales[chosenSales['Distrikt']==district]
  chosenSales = chosenSales[chosenSales['Varetype']==product]

  #Changing column names to match data frames
  products = products.rename(columns={"VMP ID": "Artikkelnr", "Alkohol %": "Alkoholprosent"})

  #Keeping interesting columns
  smallSales = chosenSales[['År_Måned','Artikkelnr','Liter denne måned i år', 'Salgspris', 'Årgang', 'Volum', 'Alkoholprosent', 'Utvalg']]
  smallRanks = ranks[['År_Måned','Artikkelnr', 'Rangering', 'Status',	'Styringstall']]
  smallProducts = products[['Artikkelnr', 'Årgang', 'Alkoholprosent', 'Emballasjetype']]

  #Converting to dask dataframe for merging
  dfSales = dd.from_pandas(smallSales, npartitions=10)
  dfRanks = dd.from_pandas(smallRanks, npartitions=10)
  dfProducts = dd.from_pandas(smallProducts, npartitions=10)

  # Merge the csv files.
  df = dd.merge(dfSales, dfRanks, how='left', on=['Artikkelnr', 'År_Måned'])
  df = dd.merge(df, dfProducts, how='left', on=['Artikkelnr','Årgang'])

  #Converting back to pandas
  df=df.compute()

  #Sorting data
  df=df.sort_values(by=['År_Måned', 'Artikkelnr', 'Årgang'])
  
  return df



###############################################################################

#Adding relevant columns and removing irrelevant rows
def column_preparation(df):
  
  #Removing articles with wrong packaging
  df = df[df['Emballasjetype']==packaging]
  
  #Summing up total amount sold per article nr and adding as column
  totalLiters=df.groupby(['Artikkelnr'])['Liter denne måned i år'].sum()
  totalLiters=totalLiters.to_dict()
  df = df.assign(Total=df['Artikkelnr'].map(totalLiters))
  
  #Using Alkoholprosent_x if not NaN, otherwise using Alkoholprosent_y
  df = df.assign(Alkoholprosent = np.where(df['Alkoholprosent_x'].notnull(), df['Alkoholprosent_x'], df['Alkoholprosent_y']) )
  df = df.drop(['Alkoholprosent_x','Alkoholprosent_y'], axis='columns')

  #Adding a column for amount of alcohol divided by sales price
  df['Alk/Pris'] = df['Alkoholprosent']*df['Volum']/df['Salgspris']  

  #Adding a column for volume divided by sales price
  df['Vol/Pris'] = df['Volum']/df['Salgspris']

  #Changing name of column to avoid space
  df = df.rename(columns={'Liter denne måned i år': 'Liter'})

  #Removing duplicates
  df = df.drop_duplicates()

  #Removing aritcles that have only 12 rows or less 
  df = df[df.groupby('Artikkelnr').Artikkelnr.transform(len) > 12]

  #Removing articles with zero total sales
  df = df[df['Total']>1000]

  #Removing articles with nan total sales
  df = df[df['Total']!=np.nan]
  
  return df


###############################################################################

#Handling duplicates

#Finding dates that are duplicated in list of dates and returning
#a string that can be used as index for dataframe
def dupDates(dates):
  seen = {}
  dupes = []

  for i in dates:
      if i not in seen:
          seen[i] = 1
      else:
          if seen[i] == 1:
              dupes.append(i)
          seen[i] += 1
  for i in range(0,len(dupes)):
    dupes[i]=str(dupes[i])
    dupes[i]=dupes[i].replace('-01T00:00:00.000000000','')
  return dupes

#Finds the first duplicate and returns article number and date.
#Helps initiate new dataframe containing only duplicated rows
def arbitrary_duplicate(df2, articles):
  if articles:
    for i in articles:
      dates=df2.loc[i].index.values
      if len(dates)-len(np.unique(dates))!=0:
        dupes=dupDates(dates)
        return i, dupes[0]
  return 0, 0


#Finding articlenrs with duplicates and returning a tuple
#with duplicated article number and date
def duplicate_index(df2,dfDupes, articles):
  dupRows= []
  for i in articles:
    dates=df2.loc[i].index.values
    if len(dates)-len(np.unique(dates))!=0:
      dupes=dupDates(dates)
      for j in range(0,len(dupes)):
        dfDupes=dfDupes.append(df2.loc[i][dupes[j]])
        d=(i,dupes[j])
        dupRows.append(d)
  return dupRows 


#Identifying the columns that vary in the duplicated rows
def nonduplicated_columns(df):
    my_cols = []
    for col in df.columns:
        if df[col].nunique(dropna=False) > 1:
            my_cols.append(list(df.columns).index(col))
    return my_cols

#Printing the columns and amount of times that column is the
#reason for duplicated rows
def duplicates():
  duplicatedColumns=np.zeros(len(df2.columns))

  for art,date in dupRows:
    #print(art, date)
    dupCols=nonduplicated_columns(df2.loc[art][date])
    for i in range(0,len(dupCols)):
      duplicatedColumns[dupCols[i]]+=1

  for i in range(0,len(duplicatedColumns)):
    print(df2.columns[i],duplicatedColumns[i])

#Duplicated rows caused by different amounts of liter sold per month is often 
#caused by multiple distributors, these rows will be handled by adding the Liter 
#columns to one row and removing the rest of the duplicated rows


def remove_duplicates(df, df2, dupRows):
  for art,date in dupRows:
      dupCols=nonduplicated_columns(df2.loc[art][date])
      indices=df.index[(df['Artikkelnr'] == art) & (df['År_Måned'] == date)].tolist()
      if 0 in dupCols:
        df.loc[indices[0],'Liter']+=df.loc[indices[1]][2]
        df=df.drop(indices[1])
      else:
        for i in range(1,len(indices)):
          df=df.drop(indices[i])
  return df


###############################################################################

#Finding difference with lag=12

#Finding difference by looping through articles.
#Is a bit slow, but doesn't take more than a few minutes. 

def differencing(df2,articles):
  #Making new column
  df2['Differanse']=0
  df2['Prosentdifferanse']=0

  #Looping through articles 
  for i in articles:
    d=df2.loc[i]['Liter'].diff(periods=12)
    pd=df2.loc[i]['Liter'].pct_change(periods=12)
    dates=df2.loc[i].index.values
    for j in range(0,len(dates)):
      df2.loc[(i, dates[j]),'Differanse']=d[dates[j]]
      df2.loc[(i, dates[j]),'Prosentdifferanse']=pd[dates[j]]
       
  return (df2.replace([np.inf, -np.inf], np.nan))


###############################################################################

#Pivoting

def pivoting(df):
  pivot = df.pivot_table(index='År_Måned', columns='Artikkelnr', aggfunc='mean' )
  pivot = pivot.reorder_levels([1,0],axis=1)
  pivot = pivot.sort_index(axis=1, ascending=True)
  return pivot


###############################################################################

def main_preparation(sales, ranks, products, country, district, product, packaging):
  df = merging(sales, ranks, products, country, district, product, packaging)
  df=column_preparation(df)

  #Preparations for analysis
  
  #List of article numbers
  articles=df['Artikkelnr'].tolist()
  articles=list(dict.fromkeys(articles))

  #List of dates, so far in wrong order
  dates=df['År_Måned'].tolist()
  dates=list(dict.fromkeys(dates))

  #Resetting index to remove problem of rows having same index
  df = df.reset_index(drop=True)
  
  #Making a copy of the dataframe with multiindex
  df2 = df.set_index(['Artikkelnr','År_Måned']).sort_index()

  #Label encoding columns to avoid pivoting issues and storing the codes in dictionaries

  #Selection
  df['Utvalg_c']= df.Utvalg.astype("category").cat.codes
  cUtvalg = df.Utvalg.astype('category')
  dUtvalg = dict(enumerate(cUtvalg.cat.categories))
  
  #Setting up new dataframe for duplicated rows, starting with arbitrary duplicate
  #print(country, district)
  dart, ddate = arbitrary_duplicate(df2, articles)

  if dart != 0:
    dfDupes=df2.loc[dart][ddate]
    
    #Tuples with duplicate articles and dates  
    dupRows=duplicate_index(df2,dfDupes, articles)
    
    #Removing duplicates
    df= remove_duplicates(df, df2, dupRows)
  
  #Making a copy of the dataframe with multiindex
  df2 = df.set_index(['Artikkelnr','År_Måned']).sort_index()
  
  #Finding difference and percentage difference
  #df2 = differencing(df2, articles)
  
  #Making a pivoted copy of df for plotting
  pivot = pivoting(df2.reset_index())
  

  return df, df2, pivot, articles, dates


#Sums up the wine sales for each month

def aggregate(column,articles,pivot):
  agg=0
  for i in articles:
    agg+=pivot[i][column].fillna(0)
  return agg


#Returns a dataframe only consisting of the top 5 wines

def top_wines(articles,df,area):
  #Constructing 2D list with total sales for each wine in this district
  artTot = [[0 for i in range(2)] for j in range(len(articles))]

  for i in range(0,len(articles)):
    artTot[i][0] = articles[i]
    artTot[i][1] = df[df['Artikkelnr'] == articles[i]].Total.values[0]

  #Sorting the values in the second column highest to lowest and keeping top n
  artTot=sorted(artTot,key=lambda x: x[1],reverse=True)[:5]

  dfPopDist = pd.DataFrame()

  if len(artTot)==5:
    #Making a dataframe consisting of only top 5 wines
    dfPopDist = df[(df['Artikkelnr'] == artTot[0][0]) |
              (df['Artikkelnr'] == artTot[1][0]) |
              (df['Artikkelnr'] == artTot[2][0]) |
              (df['Artikkelnr'] == artTot[3][0]) | 
              (df['Artikkelnr'] == artTot[4][0])]

    #Adding columns with district and country
    dfPopDist.insert(2,'Område', str(area))

  return dfPopDist

#df, df2, pivot, articles, dates=main_preparation(sales, ranks, products, country, district, product, packaging)

#Looping through each area to make two separate dataframes, one containing the 
#aggreagated liters sold per month per district and another to include the liters 
#sold per month for the five most popular wines per district  

dfAgg = pd.DataFrame()
dfPop = pd.DataFrame()
for i in range(0,len(areas)):
  df, df2, pivot, articles, dates=main_preparation(sales, ranks, products, areas[i][0], areas[i][1], product, packaging)
  
  #Aggregating the sales per month and adding new columns for district
  aggLiter = aggregate('Liter',articles,pivot)
  dfAgg[areas[i]] = aggLiter

  #Finding the top 5 wines for each district and adding them to a dataframe
  dfPopDist =  top_wines(articles,df,areas[i])
  dfPop = dfPop.append(dfPopDist)




Converting dataframes to .csv files for later analysis

In [ ]:
dfPop.to_csv('dfTopWines.csv')

In [ ]:
dfAgg.to_csv('dfAgg2.csv')